In [1]:
import pickle
import random
from math import sqrt
from statistics import mean

import pandas as pd

# Importing Datasets

In [2]:
# World Cup Matches
matches = pd.read_csv("../WorldCupMatches.csv")

# wwmatches
wwmatches = pd.read_csv("../wwmatches.csv")

# fifa_ranking
fifa_ranking = pd.read_csv("../fifa_ranking.csv")

## Load `team_players` binary

In [3]:
with open("../../site/server/match_predictor/ml_data/team_players.b", "rb") as f:
    team_players = pickle.load(f)

# Combining the matches

In [4]:
matches = matches.append(wwmatches, sort=True)
matches = matches.reset_index().drop("index", axis=1)

### Narrowing the dataset to 2018 FIFA participants

In [5]:
new_matches = pd.DataFrame(columns=matches.columns)

for i in range(len(matches)):
    if matches["Home Team Name"][i] in team_players.keys() and matches["Away Team Name"][i] in team_players.keys():
        new_matches = new_matches.append(matches.iloc[i])

    elif matches["Home Team Name"][i] == "Iran" and matches["Away Team Name"][i] in team_players.keys():
        matches["Home Team Name"][i] = "IR Iran"
        new_matches = new_matches.append(matches.iloc[i])
        
    elif matches["Away Team Name"][i] == "Iran" and matches["Home Team Name"][i] in team_players.keys():
        matches["Away Team Name"][i] = "IR Iran"
        new_matches = new_matches.append(matches.iloc[i])
    
matches = new_matches

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
matches = matches.sort_values("Year")
matches = matches.reset_index().drop("index", axis=1)

In [7]:
len(matches)

1376

In [8]:
matches.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year
0,0.0,Morocco,2.0,Nigeria,2000.0
1,0.0,Denmark,3.0,France,2000.0
2,1.0,Senegal,4.0,Tunisia,2000.0
3,2.0,England,3.0,Portugal,2000.0
4,0.0,Germany,1.0,England,2000.0


# Add `fifa_ranking`

In [9]:
fifa_ranking.head()

,rank,country,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,year
0,1,Germany,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1993
1,2,Italy,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1993
2,3,Switzerland,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1993
3,4,Sweden,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1993
4,5,Argentina,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1993


In [10]:
def find_rank(team_name, year):
    l = 0
    r = len(fifa_ranking)-1
    year_i = -1
    
    while (l < r):
        mid = (l+r)//2
        if fifa_ranking["year"][mid] > year:
            r = mid-1
        elif fifa_ranking["year"][mid] < year:
            l = mid+1
        else:
            year_i = mid
            break
            
    if year_i == -1:
        return False
    
    tmp = fifa_ranking["year"][year_i]
    while year_i >= 0 and fifa_ranking["year"][year_i] == tmp:
        year_i-=1
        
    year_i+=1
    
    for i in range(year_i, len(fifa_ranking)):
        if fifa_ranking["country"][i] == team_name:
            return fifa_ranking.iloc[i]
        
    return False

## Add necessary columns

In [11]:
columns = list(matches.columns)

home_team_rank = find_rank(matches["Home Team Name"][0], 2000)
away_team_rank = find_rank(matches["Away Team Name"][0], 2000)
        
home_rank_columns = ["home_"+x for x in home_team_rank.index]
away_rank_columns = ["away_"+x for x in away_team_rank.index]

columns += home_rank_columns
columns += away_rank_columns

In [12]:
columns

['Away Team Goals',
 'Away Team Name',
 'Home Team Goals',
 'Home Team Name',
 'Year',
 'home_rank',
 'home_country',
 'home_total_points',
 'home_previous_points',
 'home_rank_change',
 'home_cur_year_avg',
 'home_cur_year_avg_weighted',
 'home_last_year_avg',
 'home_last_year_avg_weighted',
 'home_two_year_ago_avg',
 'home_two_year_ago_weighted',
 'home_three_year_ago_avg',
 'home_three_year_ago_weighted',
 'home_year',
 'away_rank',
 'away_country',
 'away_total_points',
 'away_previous_points',
 'away_rank_change',
 'away_cur_year_avg',
 'away_cur_year_avg_weighted',
 'away_last_year_avg',
 'away_last_year_avg_weighted',
 'away_two_year_ago_avg',
 'away_two_year_ago_weighted',
 'away_three_year_ago_avg',
 'away_three_year_ago_weighted',
 'away_year']

In [13]:
new_matches = pd.DataFrame(columns=columns)

for i in range(len(matches)):
    data = list(matches.iloc[i])
    
    year = matches["Year"][i]
    home_team_rank = find_rank(matches["Home Team Name"][i], year)
    away_team_rank = find_rank(matches["Away Team Name"][i], year)
    
    home_year = year
    while home_team_rank is False:
        home_year+=1
        home_team_rank = find_rank(matches["Home Team Name"][i], home_year)
    
    away_year = year
    while away_team_rank is False:
        away_year+=1
        away_team_rank = find_rank(matches["Away Team Name"][i], away_year)
    
    data += list(home_team_rank)
    data += list(away_team_rank)
    new_matches = new_matches.append(
        pd.Series(data, index=columns),
        ignore_index=True
    )
    
matches = new_matches

In [14]:
matches.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year,home_rank,home_country,home_total_points,home_previous_points,home_rank_change,...,away_rank_change,away_cur_year_avg,away_cur_year_avg_weighted,away_last_year_avg,away_last_year_avg_weighted,away_two_year_ago_avg,away_two_year_ago_weighted,away_three_year_ago_avg,away_three_year_ago_weighted,away_year
0,0.0,Morocco,2.0,Nigeria,2000.0,76,Nigeria,0.0,444,0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000
1,0.0,Denmark,3.0,France,2000.0,3,France,0.0,765,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000
2,1.0,Senegal,4.0,Tunisia,2000.0,28,Tunisia,0.0,596,3,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000
3,2.0,England,3.0,Portugal,2000.0,15,Portugal,0.0,672,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000
4,0.0,Germany,1.0,England,2000.0,12,England,0.0,695,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000


In [15]:
matches.columns

Index(['Away Team Goals', 'Away Team Name', 'Home Team Goals',
       'Home Team Name', 'Year', 'home_rank', 'home_country',
       'home_total_points', 'home_previous_points', 'home_rank_change',
       'home_cur_year_avg', 'home_cur_year_avg_weighted', 'home_last_year_avg',
       'home_last_year_avg_weighted', 'home_two_year_ago_avg',
       'home_two_year_ago_weighted', 'home_three_year_ago_avg',
       'home_three_year_ago_weighted', 'home_year', 'away_rank',
       'away_country', 'away_total_points', 'away_previous_points',
       'away_rank_change', 'away_cur_year_avg', 'away_cur_year_avg_weighted',
       'away_last_year_avg', 'away_last_year_avg_weighted',
       'away_two_year_ago_avg', 'away_two_year_ago_weighted',
       'away_three_year_ago_avg', 'away_three_year_ago_weighted', 'away_year'],
      dtype='object')

In [16]:
matches.drop(["home_country", "home_year", "away_country", "away_year"], axis=1, inplace=True)

## Save matches into CSV file (final.csv)

In [17]:
matches.to_csv("../final.csv", index=0)